# Reinforcement Learning
Reinforcement learning is a machine learning training method based on rewarding desired behaviors and/or punishing undesired ones.

In general, a reinforcement learning agent is able to perceive and interpret its environment, take actions and learn through trial and error.

<img src="https://raw.githubusercontent.com/MandM-DataScience/chess-engine/master/images/Reinforcement_learning_diagram.svg.png" alt="Reinforcement learning diagram" width=300 height=300>

The agent takes in a representation of the environment (State) and decides which Action to take (based on what it thinks the reward will be).

At the beginning it has no idea of which Action will yield the best reward, so it will take random Actions and discover step by step which Action is the best for each State!

This is the trial and error part of the process.

So, the main components of our reinforcement learning model are:
- State representation (how are we going to encode a chess board position into a State)
- Action space (what possible Action can the agent make)
- Neural network architecture (how the Agent chooses which Action to take based on the current State)
- Reward signal (how are we going to reward/punish the agent for an Action it took)
- Exploration strategy (how do we balance the exploration of new Actions with the exploitation of what the Agent has learned in previous steps)

## State Representation

A chess board position can be efficiently stored as a list of bitboards.

A bitboard is a specialized bit array data structure commonly used in computer systems that play board games, where each bit corresponds to a game board space or piece.

To represent the whole board position using bitboards we need one bitboard for each piece type + one bitboard for the empty squares.

Let's see an example with the starting board.

In [13]:
import chess
board = chess.Board()
print(board.unicode())

♜ ♞ ♝ ♛ ♚ ♝ ♞ ♜
♟ ♟ ♟ ♟ ♟ ♟ ♟ ♟
⭘ ⭘ ⭘ ⭘ ⭘ ⭘ ⭘ ⭘
⭘ ⭘ ⭘ ⭘ ⭘ ⭘ ⭘ ⭘
⭘ ⭘ ⭘ ⭘ ⭘ ⭘ ⭘ ⭘
⭘ ⭘ ⭘ ⭘ ⭘ ⭘ ⭘ ⭘
♙ ♙ ♙ ♙ ♙ ♙ ♙ ♙
♖ ♘ ♗ ♕ ♔ ♗ ♘ ♖


In [14]:
def print_bitboard(bitboard):
    
    # for each row
    for i in range(8):
        
        # print bitboard of 1s/0s (we have to mirror the bitboard)
        if i == 0:
            print(format(bitboard, "064b")[8*(i+1)-1::-1])
        else:
            print(format(bitboard, "064b")[8*(i+1)-1:8*i-1:-1])

# bitboard of pawns
print_bitboard(board.pawns)

00000000
11111111
00000000
00000000
00000000
00000000
11111111
00000000


In [15]:
# bitboard of kings
print_bitboard(board.kings)

00001000
00000000
00000000
00000000
00000000
00000000
00000000
00001000


In [16]:
# bitboard of bishops
print_bitboard(board.bishops)

00100100
00000000
00000000
00000000
00000000
00000000
00000000
00100100


In [17]:
# bitboard of empty squares
print_bitboard(board.occupied ^ 2 ** 64 -1)

00000000
00000000
11111111
11111111
11111111
11111111
00000000
00000000


## Action Space
The Action space must contain all POSSIBLE moves, not VALID moves.

We decided the number of possible moves as 4096 (64 squares from -> 64 squares to).

Out of this 4096 combinations, not all of them are truly possible (for example no piece can move from square a1 to square b7). But for this exercise we decide to ignore this as it would complicate things.

Out of this set of possible moves, of course only a very small subset of moves can be chosen in a specific state.

To narrow down these valid moves we are going to apply a masking layer in our Neural Net.

In [18]:
import numpy as np

def encode_valid_moves(board):
    
    valid_moves_dict = {}
    
    # for each valid move
    for move in board.legal_moves:
        
        # compute index based on starting square and target square
        index = 64 * (move.from_square) + (move.to_square)
        valid_moves_dict[index] = move
    
    return valid_moves_dict

valid_moves_dict = encode_valid_moves(board)
valid_moves_dict

{407: Move.from_uci('g1h3'),
 405: Move.from_uci('g1f3'),
 82: Move.from_uci('b1c3'),
 80: Move.from_uci('b1a3'),
 983: Move.from_uci('h2h3'),
 918: Move.from_uci('g2g3'),
 853: Move.from_uci('f2f3'),
 788: Move.from_uci('e2e3'),
 723: Move.from_uci('d2d3'),
 658: Move.from_uci('c2c3'),
 593: Move.from_uci('b2b3'),
 528: Move.from_uci('a2a3'),
 991: Move.from_uci('h2h4'),
 926: Move.from_uci('g2g4'),
 861: Move.from_uci('f2f4'),
 796: Move.from_uci('e2e4'),
 731: Move.from_uci('d2d4'),
 666: Move.from_uci('c2c4'),
 601: Move.from_uci('b2b4'),
 536: Move.from_uci('a2a4')}

Each one of the 4096 moves represent a move from a square to another. The first move (which is not actually possible) is move from a1 to a1. The second a1 -> b1, the third a1 -> c1, and so on.

The index grows for the target square from left to right (by 1) and from bottom to top (by 8).

The index grows for the starting square from left to right (by 64) and from bottom to top (by 512).

With these moves encoded in this way we can build a mask (of 0s/1s) in order to disable the output layer nodes that correspond to a non-valid move.

In [19]:
def mask_and_valid_moves(board):

    mask = np.zeros((64, 64))
    valid_moves_dict = {}
    
    # for each valid move
    for move in board.legal_moves:
        
        # mask is a matrix
        mask[move.from_square, move.to_square] = 1
        
        # compute index based on starting square and target square
        index = 64 * (move.from_square) + (move.to_square)
        
        valid_moves_dict[index] = move
    
    return mask, valid_moves_dict

mask, valid_moves_dict = mask_and_valid_moves(board)
mask

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Neural Network architecture
We are going to use an hybrid neural network consisting of an initial series of convolutional layers and a following series of fully connected layers.

The convolutional layers are used to extract features from the "image" of the board, the fully connected layers to compute a function that takes this features as input and the move to make as output.

For the final step we will use a mask layer to reduce the number of selectable moves to only the valid moves.

The process is quite similar to how a human would act.

A human looks at the board and extract features like piece position, possible attacks, threats from the opponent, possible moves, checks, and so on. And then uses this information to take a decision about which move to make.

### How does a Neural network work?
A neural network is made of layers and nodes.

<img src="https://raw.githubusercontent.com/MandM-DataScience/chess-engine/master/images/colored_neural_network-1200x1443.png" alt="Neural network diagram" width=250 height=250>

Each node of a layer takes as input the output of the previous layer (or a subset of that) and computes a function on that input (activation function).

Each node has a weight and bias parameters that are used inside that function.

The output of the nodes of the layer is then passed as input to the next layer and so on, until the final layer that will output the neural net prediction.

The prediction is then compared to the actual values, computing a loss (or error).

This error is then backpropagated from the output layer to previous layers, where each node adjust its weight and bias parameters in order to minimize that error.

![Neural network backpropagation](https://raw.githubusercontent.com/MandM-DataScience/chess-engine/master/images/backpropagation.gif)

### How does a convolutional layer work?
The layer performs a dot product between two matrices, where one matrix is the set of learnable parameters otherwise known as a kernel, and the other matrix is a subset of the input.

<img src="https://raw.githubusercontent.com/MandM-DataScience/chess-engine/master/images/convolution.gif" alt="Convolution" width=400 height=400>

During the forward pass, the kernel slides across the height and width of the image/input. The sliding size of the kernel is called a stride.

This is used to extract features from subsets of the image to be passed to following layers.

Let's see how to build this network using PyTorch, a library used to create neural networks in python. https://pytorch.org/

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# This is our final mask layer
class MaskLayer(nn.Module):

    def __init__(self):
        super(MaskLayer, self).__init__()
    
    # mask is made of 0s/1s so it will just set to 0 any invalid move
    def forward(self, x, mask):
        return torch.mul(x, mask)

BatchNorm2d layers are normalization layers that subtract the batch mean and divide by the batch standard deviation.

It then applies a scaling factor and an offset to the normalized output. The scaling factor and offset are learned during training and are applied during both training and inference.

The purpose of these layers is to improve the training process and reduce overfitting by normalizing the input to each node in a layer. This can lead to faster convergence during training and better generalization to new data during inference.

In [21]:
class DQN(nn.Module):

    def __init__(self):
        
        super(DQN, self).__init__()
        
        # input size = 8 (rows) x 8 (cols) x 16 (bitboards)
        # - 6 bitboards for white pieces
        # - 6 bitboards for black pieces
        # - 1 for empty squares
        # - 1 for castling rights
        # - 1 for en passant
        # - 1 for player
        
        # first convolutional layer 8x8x16 => 8x8x32
        self.conv1 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        
        # second convolutional layer 8x8x32 => 8x8x64 
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        
        # third convolutional layer 8x8x64 => 8x8x128 
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        # first fully connected layer 8192 => 8192
        self.fc1 = nn.Linear(128*64, 128*64)
        
        # second fully connected layer 8192 => 4096
        self.fc2 = nn.Linear(128*64, 64*64)
        
        # mask is made of 0s/1s so it will just set to 0 any invalid move 4096 => 4096
        self.mask = MaskLayer()

    def forward(self, x, mask=None, debug=False):
        
        # conv1 + bn1 with activation function ReLU
        x = nn.functional.relu(self.bn1(self.conv1(x)))
        
        # conv2 + bn2 with activation function ReLU
        x = nn.functional.relu(self.bn2(self.conv2(x)))
        
        # conv3 + bn3 with activation function ReLU
        x = nn.functional.relu(self.bn3(self.conv3(x)))
        
        # flatten will transform data structure from 3D 8x8x128 to 1D 8192
        x = nn.Flatten()(x)
        
        # fully connected with activation function ReLU
        x = nn.functional.relu(self.fc1(x))
        
        # fully connected WITHOUT ReLU (we want to keep negative values for our output layer)
        x = self.fc2(x)
        
        # if we have a mask we apply it to set to 0 all invalid moves
        if mask is not None:
            x = self.mask(x, mask)
            
        return x

A rectified linear unit (ReLU) is an activation function that introduces the property of non-linearity to a neural network model and solves the vanishing gradients issue.

It is one of the most popular activation functions and it works like a floor function, max(0, output).

https://en.wikipedia.org/wiki/Rectifier_(neural_networks)

## Reward signal
The main signals should be the ones regarding the result of a match: +1 if win, -1 if lose, 0 if draw.

However, it is better to introduce intermediate signals, so that we can "teach" the agent also during a single game and not only at the end.

For example a small reward if by making a move the position improves and a lower score if the position worsen.

Keep in mind that to assess the position score in the middle of a game we would need something "external" from the model! In our case we will use the stockfish analyse function.

It is definitevely possible to train a model without any kind of human or other-engine intervention, and just using the match result rewards. But it will probably take longer.

In [24]:
import chess
import chess.engine
import random

# for now our agent choose a random move
def agent_choose_move(board):
    return random.choice(list(board.legal_moves))

# Create a chess board
board = chess.Board()

# Create a stockfish engine instance
stockfish = chess.engine.SimpleEngine.popen_uci("/Users/simarjeetss529/Desktop/saar/uni/semesters/spr 25/cpts 540 ai/project/CPT_S-440/stockfish/stockfish-macos-m1-apple-silicon")

# Analyse starting board with stockfish
board_score_before = stockfish.analyse(board=board, limit=chess.engine.Limit(depth=5))\
    ['score'].relative.score(mate_score=10000)

# Agent choose move
move = agent_choose_move(board)
board.push(move)

# Make random move for black
board.push(random.choice(list(board.legal_moves)))

# Analyse final board with stockfish
board_score_after = stockfish.analyse(board=board, limit=chess.engine.Limit(depth=5))\
    ['score'].relative.score(mate_score=10000)

# Divide by 100 to transform to centipawn to pawn score and subtract 0.01 to penalize the agent for each move. 
# We want to win as fast as possible ;)
reward = board_score_after/100 - board_score_before/100 - 0.01

In [25]:
print(board.unicode())

♜ ♞ ♝ ♛ ♚ ♝ ♞ ♜
⭘ ♟ ♟ ♟ ♟ ♟ ♟ ♟
♟ ⭘ ⭘ ⭘ ⭘ ⭘ ⭘ ⭘
⭘ ⭘ ⭘ ⭘ ⭘ ⭘ ⭘ ⭘
⭘ ♙ ⭘ ⭘ ⭘ ⭘ ⭘ ⭘
⭘ ⭘ ⭘ ⭘ ⭘ ⭘ ⭘ ⭘
♙ ⭘ ♙ ♙ ♙ ♙ ♙ ♙
♖ ♘ ♗ ♕ ♔ ♗ ♘ ♖


In [28]:
move

Move.from_uci('b2b4')

In [29]:
reward

-0.18

## Exploration strategy
It is crucial to design the agent with an exploration strategy that will allow it to explore new moves instead of picking always the ones that temporarily thinks as best.

Otherwise, the agent will become stuck in some local minimum instead of getting to the best possible solution.

A simple exploration strategy is called epsilon-greedy.

We have a parameter called epsilon that ranges between 0 and 1 and represent the exploration probability.

At the beginning of the training the epsilon is set to a high value, like 1 and it is progressively reduced.

For each step the agent takes a random action with probability epsilon and the "best move" with probability 1 - epsilon.

In [30]:
# Exploration rate
epsilon = 1
epsilon_decay = 0.995
epsilon_min = 0.01

# choose random with probability epsilon
if random.uniform(0, 1) <= epsilon:
    move = random.choice(list(board.legal_moves))
    
# choose best move with probability 1 - epsilon
else:
    move = agent_choose_move(board)

# reduce exploration rate after each step
epsilon = max(epsilon * epsilon_decay, epsilon_min)
epsilon

0.995

## Let's build the agent

In [31]:
import os

class ChessAgent:

    # Constructor
    def __init__(self, input_model_path=None):

        # Exploration parameters
        self.epsilon = 1
        self.epsilon_decay = 0.99
        self.epsilon_min = 0.01

        # Training parameters
        self.gamma = 0.5 # tells the agent whether to prefer long term rewards or immediate rewards. 0 = greedy, 1 = long term
        self.learning_rate = 1e-03 # how fast the network updates its weights
        self.MEMORY_SIZE = 512 # how many steps/moves/samples to store. It is used for training (experience replay) 
        self.MAX_PRIORITY = 1e+06 # max priority for a sample in memory. The higher the priority, the more likely the sample will be included in training
        self.memory = [] # memory data structure
        self.batch_size = 16 # how many sample to include in a training step        
        
        self.policy_net = DQN()
        
        # Load trained model if exists
        if input_model_path is not None and os.path.exists(input_model_path):
            self.policy_net.load_state_dict(torch.load(input_model_path))    

        # We use mean squared error as our loss function
        self.loss_function = nn.MSELoss()
        
        # Adam optimizer provides adaptive learning rate and a momentum-based approach that can help the neural network 
        # learn faster and converge more quickly towards the optimal set of parameters that minimize the cost or loss function
        self.optimizer = torch.optim.Adam(self.policy_net.parameters(), lr=self.learning_rate)

    
    # Convert board into a 3D np.array of 16 bitboards
    def convert_state(self, board):
        
        # dictionary to store bitboards
        piece_bitboards = {}
        
        # for each color (white, black)
        for color in chess.COLORS:
            
            # for each piece type (pawn, bishop, knigh, rook, queen, kinb)
            for piece_type in chess.PIECE_TYPES:
                v = board.pieces_mask(piece_type, color)
                symbol = chess.piece_symbol(piece_type)
                i = symbol.upper() if color else symbol
                piece_bitboards[i] = v

        # empty bitboard
        piece_bitboards['-'] = board.occupied ^ 2 ** 64 - 1
        
        # player bitboard (full 1s if player is white, full 0s otherwise)
        player = 2 ** 64 - 1 if board.turn else 0

        # castling_rights bitboard
        castling_rights = board.castling_rights

        # en passant bitboard
        en_passant = 0
        ep = board.ep_square
        if ep is not None:
            en_passant |= (1 << ep)

        # bitboards (16) = 12 for pieces, 1 for empty squares, 1 for player, 1 for castling rights, 1 for en passant
        bitboards = [b for b in piece_bitboards.values()] + [player] + [castling_rights] + [en_passant]

        # for each bitboard transform integet into a matrix of 1s and 0s
        # reshape in 3D format (16 x 8 x 8)
        bitarray = np.array([
            np.array([(bitboard >> i & 1) for i in range(64)])
            for bitboard in bitboards
        ]).reshape((16, 8, 8))

        return bitarray

    
    # get the move index out of the 4096 possible moves, as explained before
    def get_move_index(self, move):
        index = 64 * (move.from_square) + (move.to_square)
        return index

    
    # returns mask of valid moves (out of 4096) + the dictionary with the valid moves and their indexes
    def mask_and_valid_moves(self, board):

        mask = np.zeros((64, 64))
        valid_moves_dict = {}
        
        for move in board.legal_moves:
            mask[move.from_square, move.to_square] = 1
            valid_moves_dict[self.get_move_index(move)] = move
        
        # mask is flatten and returned as a PyTorch tensor
        # a tensor is just a vector optimized for derivatives computation, used in PyTorch neural nets
        return torch.from_numpy(mask.flatten()), valid_moves_dict

    
    # insert a step/move/sample into memory to be used in training as experience replay
    def remember(self, priority, state, action, reward, next_state, done, valid_moves, next_valid_moves):

        # if memory is full, we delete the least priority element
        if len(self.memory) >= self.MEMORY_SIZE:
            
            min_value = self.MAX_PRIORITY
            min_index = 0
            
            for i,n in enumerate(self.memory):
                
                # priority is stored in the first position of the tuple
                if n[0] < min_value:
                    min_value = n[0]
                    min_index = i
            
            del self.memory[min_index]

        self.memory.append((priority, state, action, reward, next_state, done, valid_moves, next_valid_moves))

    
    # Take a board as input and return a valid move defined as tuple (start square, end square)
    def select_action(self, board, best_move):

        # convert board into the 16 bitboards
        bit_state = self.convert_state(board)
        
        # get valid moves
        valid_moves_tensor, valid_move_dict = self.mask_and_valid_moves(board)
        
        # with probability epsilon = Explore
        if random.uniform(0, 1) <= self.epsilon:
            
            r = random.uniform(0, 1)
            
            # inside exploration with probability 10% choose best move (as computed by stockfish)
            if r <= 0.1:
                chosen_move = best_move
            
            # with probability 90% choose a random move
            else:
                chosen_move = random.choice(list(valid_move_dict.values()))
        
        # with probability 1 - epsilon = Exploit
        else:
            
            # during inference we don't need to compute gradients
            with torch.no_grad():
                
                # transform our 16 bitboards in a tensor of shape 1 x 16 x 8 x 8
                tensor = torch.from_numpy(bit_state).float().unsqueeze(0)
                
                # predict rewards for each valid move in the current state. valid_moves_tensor is the mask!
                policy_values = self.policy_net(tensor, valid_moves_tensor)
                
                # take the move index with the highest predicted reward
                chosen_move_index = int(policy_values.max(1)[1].view(1,1))
                
                # if move is valid:
                if chosen_move_index in valid_move_dict:
                    chosen_move = valid_move_dict[chosen_move_index]
                    
                # if move is NOT valid, choose random move
                # this can happen if all valid moves have predicted values 0 or negative
                else:
                    chosen_move = random.choice(list(board.legal_moves))

        return self.get_move_index(chosen_move), chosen_move, bit_state, valid_moves_tensor

    
    # Decay epsilon (exploration rate)
    def adaptiveEGreedy(self):
        self.epsilon = max(self.epsilon * self.epsilon_decay, self.epsilon_min)

        
    # Save trained model
    def save_model(self, path):
        torch.save(self.policy_net.state_dict(), path)